In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# Pobranie danych
url = "https://raw.githubusercontent.com/selva86/datasets/master/daily-min-temperatures.csv"
df = pd.read_csv(url, parse_dates=['Date'])

# Wybór tylko temperatury
data = df['Temp'].values.astype(np.float32)

# Normalizacja danych
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.reshape(-1, 1))


In [ ]:
def create_sequences(data, seq_length=5):
    x, y = [], []
    for i in range(len(data) - seq_length):
        x.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(x), np.array(y)

x, y = create_sequences(data_scaled, seq_length=5)

# Podział na zbiór treningowy i testowy (80/20)
split = int(len(x) * 0.8)
x_train, x_test = x[:split], x[split:]
y_train, y_test = y[:split], y[split:]

# Konwersja na tensory PyTorch
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [ ]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(1, 32, batch_first=True)
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

model = LSTM()


In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Dodajemy wymiar kanału (LSTM wymaga 3D wejścia)
x_train = x_train.unsqueeze(-1)
x_test = x_test.unsqueeze(-1)

# Trening modelu (5 epok – krótki czas wykonania)
for epoch in range(5):
    for i in range(len(x_train)):
        optimizer.zero_grad()
        y_pred = model(x_train[i:i+1])
        loss = loss_fn(y_pred, y_train[i:i+1])
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(x_test).numpy()

# Odwracamy skalowanie danych
y_test = scaler.inverse_transform(y_test.numpy().reshape(-1, 1))
predictions = scaler.inverse_transform(predictions)

# Wykres
plt.plot(y_test[:100], label="Rzeczywiste")
plt.plot(predictions[:100], label="Prognozy")
plt.legend()
plt.show()
